# EDA and Data Loading

This notebook covers:
- Exploratory Data Analysis
- Data ingestion on Databricks
- Initial data quality checks


In [ ]:
# Import data loader
from src.data_loader import load_cpg_data
import pandas as pd

# Load data (PySpark or pandas depending on environment)
# df = load_cpg_data('data/cpg_sales_data.parquet')

# Basic EDA
# print(df.head())
# print(df.info())
# print(df.describe())


In [ ]:
# Data quality checks
# Check for missing values
# Check for duplicates
# Check data types
# Visualize distributions


# Exploratory Data Analysis and Data Loading

This notebook covers:
- Loading CPG sales data from parquet format
- Initial exploratory data analysis
- Data ingestion setup for Databricks


In [ ]:
# Import necessary libraries
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from src.data_loader import load_data

# Initialize Spark session (if using Databricks)
# spark = SparkSession.builder.appName("CPG_EDA").getOrCreate()

# Load data
# df = load_data("data/cpg_sales_data.parquet")


In [ ]:
# EDA: Basic statistics
# df.describe()
# df.info()
# df.head()
